<a href="https://colab.research.google.com/github/born99/Video-Summarization/blob/master/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pwd
import os
os.chdir('/content/drive/My Drive/Video-Summarization')

!pip install chainer

/content/drive/My Drive/Video-Summarization
     |████████████████████████████████| 1.0 MB 4.3 MB/s 
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967740 sha256=bafef32c26d081688f5defb810bb7979900dd9c934b657dbf466b8d63d719e04
  Stored in directory: /root/.cache/pip/wheels/c8/6a/6f/fd563166cc597e5206e375ea074ea836e5db5dd58421215672
Successfully built chainer


In [14]:
import numpy as np
import os
import json

from func.nets import vid_enc, vid_enc_vgg19
from chainer import serializers

import sys
sys.path.append('script/')
from summarize import get_flabel
from func.sampling.vsum import VSUM

import chainer
from chainer import configuration

In [15]:
# settings
feat_type = 'smt_feat' # smt_feat (proposed) or vgg

# load embedding model
if feat_type == 'smt_feat':
    model = vid_enc.Model()
    serializers.load_npz('data/trained_model/model_par', model)
elif feat_type == 'vgg':
    model = vid_enc_vgg19.Model()
else:
    raise RuntimeError('[invalid feat_type] use smt_feat or vgg')

FileNotFoundError: ignored

In [ ]:
# prepair output dir
d_name = 'summe'
dataset_root = 'data/{}/'.format(d_name)
out_dir = 'results/{:}/{:}/'.format(d_name, feat_type)
print ('save to: ', out_dir)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

save to:  results/summe/smt_feat/


In [ ]:
# load dataset metadata
dataset = json.load(open(dataset_root + 'dataset.json'))
video_id = [d['videoID'] for d in dataset]

print ('Video list:')
for vi in video_id:
    print ('-', vi)

Video list:
- Kids_playing_in_leaves
- playing_ball
- Paintball
- Cooking
- Scuba
- Cockpit_Landing
- Fire Domino
- Air_Force_One
- Bus_in_Rock_Tunnel
- Base jumping
- Jumps
- Saving dolphines
- Bearpark_climbing
- Uncut_Evening_Flight
- Valparaiso_Downhill
- Eiffel Tower
- Excavators river crossing
- St Maarten Landing
- Playing_on_water_slide
- Statue of Liberty
- Car_railcrossing
- paluma_jump
- Notre_Dame
- car_over_camera
- Bike Polo


In [ ]:
# summarize video
v_id = 'Cooking'

with configuration.using_config('train', False):
    with chainer.no_backprop_mode():
        vsum = VSUM(v_id, model, dataset=d_name, seg_l=5)

_, frames, _ = vsum.summarizeRep(seg_l=5, weights=[1.0, 0.0])

# get 0/1 label for each frame
fps = vsum.dataset.data['fps']
fnum = vsum.dataset.data['fnum']
label = get_flabel(frames, fnum, fps, seg_l=5)

load Cooking


TypeError: 'function' object is not subscriptable

In [ ]:
# write summarized video
from skvideo.io import vread, vwrite

video_path = 'C:/Users/admin/Desktop/vsum_dsf/data/summe/SumMe/videos/%s.webm' % v_id

video_data = vread(video_path)
#sum_vid = video_data[label.ravel().astype(np.bool), :,:,:]

print ('writing video to', 'sum_%s.mp4'%v_id)
vwrite('sum_%s.mp4'%v_id, video_data)

writing video to sum_Cooking.mp4
